In [ ]:
import os
import sys
import pandas as pd
import datetime
import numpy as np
import warnings
warnings.filterwarnings('ignore') 
#import xlsxwriter
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from dateutil.relativedelta import relativedelta


In [ ]:
import pandas_gbq

In [ ]:
import numpy_financial as npf

## Ferratum AU Oct 2021

In [ ]:
###Manual: adjust book_id and region
book_id='3435712288'
region='au' ###'au' or 'eu'

In [ ]:
query_purchase='''SELECT book_id,actual_purchase_price*-1 as purchase_local_ccy,settlement_date,actual_purchase_price_usd
                FROM presentation.dim_books db
                where 1=1
                and book_id='{}' '''.format(book_id)

In [ ]:
df_purch = pandas_gbq.read_gbq(query_purchase,project_id="data-team-318806")

In [ ]:
if region=='au':
    query_cf='''select * except (market, dca_name) from
            presentation.collections_au
            where 1=1
            and book_id='{}'  '''.format(book_id)
elif region=='eu':
    query_cf='''select * except (market, dca_name) from
            presentation.collections_eu
            where 1=1
            and book_id='{}'  '''.format(book_id)
else:
    query_cf=None


In [ ]:
df_cf=pandas_gbq.read_gbq(query_cf,project_id="data-team-318806")

In [ ]:
df_cf_agg = df_cf.groupby(['book_id', 'book_name']).sum().reset_index().copy()

In [ ]:
####Removing as of 3rd row the zero values until first non zero
# Find the index of the first non-zero column starting from the third column
non_zero_columns = df_cf_agg.columns[2:]  # Only columns from C onwards
first_non_zero_col = next((col for col in non_zero_columns if df_cf_agg[col].iloc[0] != 0), None)

# Identify columns to drop: from the third column up to the first non-zero column
if first_non_zero_col:
    columns_to_drop = []
    for col in non_zero_columns:
        if col == first_non_zero_col:
            break
        if df_cf_agg[col].iloc[0] == 0.0:
            columns_to_drop.append(col)
    df_cf_agg.drop(columns=columns_to_drop, inplace=True)

In [ ]:
book_name=df_cf_agg.iloc[0,1]

In [ ]:
cf_values = df_cf_agg.iloc[0, 2:].tolist()

In [ ]:
relevant_months=df_cf_agg.columns[2:].tolist()

In [ ]:
df_agg_cf_format=pd.DataFrame({'collection':cf_values,'month':relevant_months})

In [ ]:
df_agg_cf_format['month_cohort']=range(1, len(df_agg_cf_format)+1)

In [ ]:
cashflow=[df_purch.iloc[0,1]]+df_agg_cf_format['collection'].tolist()

In [ ]:
cashflow=[round(value, 2) for value in cashflow]

In [ ]:
####Calculating dynamic IRR
irr=[]
selected_values=[cashflow[0]]
for i in range(1, len(cashflow)):
    selected_values.append(cashflow[i])
    irr_temp = round(npf.irr(selected_values),4)
    irr.append(irr_temp)
    
df_agg_cf_format['irr']=irr

In [ ]:
#### Calculating Dynamic MOIC
moic_num=0.0
moic_temp=0.0
moic=[]
purchase_value=abs(df_purch.iloc[0,1])
selected_values=[]
for i in range(1, len(cashflow)):
    selected_values.append(cashflow[i])
    moic_num = round((sum(selected_values)),2)
    moic_temp=round(moic_num/purchase_value,2)
    moic.append(moic_temp)
    
df_agg_cf_format['moic']=moic 

In [ ]:
df_agg_cf_format['book_id']=book_id
df_agg_cf_format['book_name']=book_name
df_agg_cf_format['purchase_local_ccy']=abs(df_purch.iloc[0,1])
df_agg_cf_format['region']=region.upper()
df_agg_cf_format['purchase_usd']=round(df_purch.iloc[0,3],2)

In [ ]:
df_agg_cf_format.head()

In [ ]:
plt.figure(figsize=(10, 6))  # Make the figure wider
plt.scatter(df_agg_cf_format['month_cohort'], df_agg_cf_format['irr'], color='green',s=12)

# Add a line connecting the dots
plt.plot(df_agg_cf_format['month_cohort'], df_agg_cf_format['irr'], color='green', linestyle='-', linewidth=1)

# Add labels and title
plt.xlabel('month Cohort')
plt.ylabel('IRR',fontsize=14)
plt.suptitle(book_name, fontsize=12)
plt.title('Capex (USD) = {:,.0f} '.format(round(df_purch.iloc[0,3]),0), fontsize=10)

# Rotate x-axis labels for better readability (optional)

#x_labels = df_agg_cf_format['month_cohort'].dt.strftime('%Y-%m-%d')  # Format as string
#short_x_labels = x_labels.str.slice(0, 7)  # Get the first 6 letters

plt.xticks(df_agg_cf_format['month_cohort'], rotation=45)


# Annotate each point with its y-value 
#for i, value in enumerate(df_agg_cf_format['irr']):
#    plt.text(df_agg_cf_format['month_cohort'][i], df_agg_cf_format['irr'][i], 
#             f'{value:,.2f}', ha='center', va='bottom', fontsize=9)


#plt.axhline(y=df_average, color='red', linestyle='--', label=f'Average: {df_average:,.2f}')

#plt.text(df_agg_cf_format['month_cohort'].min(), df_average, f'Avg: {df_average:,.2f}', 
 #        color='red', va='bottom', ha='right', fontsize=10)

# Get the last value for annotation
last_x = df_agg_cf_format['month_cohort'].iloc[-1]
last_y = df_agg_cf_format['irr'].iloc[-1]
# Add a label for the last point
plt.text(last_x, last_y, f'{last_y:.2f}', ha='center', va='bottom', fontsize=9, color='green')

# Add a line where y = 0
plt.axhline(y=0, color='red', linestyle='--', label='y = 0')
# Add a label for the y = 0 line
plt.text(df_agg_cf_format['month_cohort'].min(), 0, '0', color='red', va='bottom', ha='center', fontsize=10)


# Add a line where y = 0.48
plt.axhline(y=0.48, color='gray', linestyle='--', label='y = 0.48')
# Add a label for the y = 0 line
plt.text(df_agg_cf_format['month_cohort'].min(), 0.48, 'IRR=48%', color='gray', va='bottom', ha='center', fontsize=10)

# Show plot
plt.tight_layout()  
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))  # Make the figure wider
plt.scatter(df_agg_cf_format['month_cohort'], df_agg_cf_format['moic'], color='blue',s=12)

# Add a line connecting the dots
plt.plot(df_agg_cf_format['month_cohort'], df_agg_cf_format['moic'], color='blue', linestyle='-', linewidth=1)

# Add labels and title
plt.xlabel('month Cohort')
plt.ylabel('MOIC',fontsize=14)
plt.suptitle(book_name, fontsize=12)
plt.title('Capex (USD) = {:,.0f} '.format(round(df_purch.iloc[0,3]),0), fontsize=10)

# Rotate x-axis labels for better readability (optional)

#x_labels = df_agg_cf_format['month_cohort'].dt.strftime('%Y-%m-%d')  # Format as string
#short_x_labels = x_labels.str.slice(0, 7)  # Get the first 6 letters

plt.xticks(df_agg_cf_format['month_cohort'], rotation=45)


# Annotate each point with its y-value 
#for i, value in enumerate(df_agg_cf_format['moic']):
#    plt.text(df_agg_cf_format['month_cohort'][i], df_agg_cf_format['moic'][i], 
#            f'{value:,.2f}', ha='center', va='bottom', fontsize=9)


#plt.axhline(y=df_average, color='red', linestyle='--', label=f'Average: {df_average:,.2f}')

#plt.text(df_agg_cf_format['month_cohort'].min(), df_average, f'Avg: {df_average:,.2f}', 
 #        color='red', va='bottom', ha='right', fontsize=10)

# Get the last value for annotation
last_x = df_agg_cf_format['month_cohort'].iloc[-1]
last_y = df_agg_cf_format['moic'].iloc[-1]
# Add a label for the last point
plt.text(last_x, last_y, f'{last_y:.2f}', ha='center', va='bottom', fontsize=9,color='blue')

# Add a line where y = 0
plt.axhline(y=0, color='red', linestyle='--', label='y = 0')
# Add a label for the y = 0 line
plt.text(df_agg_cf_format['month_cohort'].min(), 0, '0', color='red', va='bottom', ha='center', fontsize=10)


# Add a line where y = 2.25
plt.axhline(y=2.25, color='gray', linestyle='--', label='y = 2.25')
# Add a label for the y = 0 line
plt.text(df_agg_cf_format['month_cohort'].min(), 2.25, 'MOIC=2.25', color='gray', va='bottom', ha='center', fontsize=10)

# Show plot
plt.tight_layout()  
plt.show()

## Primagas ES Jan 2023

In [ ]:
###Manual: adjust book_id and region
book_id='3791494045'
region='eu' ###'au' or 'eu'

In [ ]:
query_purchase='''SELECT book_id,actual_purchase_price*-1 as purchase_local_ccy,settlement_date,actual_purchase_price_usd
                FROM presentation.dim_books db
                where 1=1
                and book_id='{}' '''.format(book_id)
				
df_purch = pandas_gbq.read_gbq(query_purchase,project_id="data-team-318806")

if region=='au':
    query_cf='''select * except (market, dca_name) from
            presentation.collections_au
            where 1=1
            and book_id='{}'  '''.format(book_id)
elif region=='eu':
    query_cf='''select * except (market, dca_name) from
            presentation.collections_eu
            where 1=1
            and book_id='{}'  '''.format(book_id)
else:
    query_cf=None


df_cf=pandas_gbq.read_gbq(query_cf,project_id="data-team-318806")

df_cf_agg = df_cf.groupby(['book_id', 'book_name']).sum().reset_index().copy()

In [ ]:
####Removing as of 3rd row the zero values until first non zero
# Find the index of the first non-zero column starting from the third column
non_zero_columns = df_cf_agg.columns[2:]  # Only columns from C onwards
first_non_zero_col = next((col for col in non_zero_columns if df_cf_agg[col].iloc[0] != 0), None)

# Identify columns to drop: from the third column up to the first non-zero column
if first_non_zero_col:
    columns_to_drop = []
    for col in non_zero_columns:
        if col == first_non_zero_col:
            break
        if df_cf_agg[col].iloc[0] == 0.0:
            columns_to_drop.append(col)
    df_cf_agg.drop(columns=columns_to_drop, inplace=True)

In [ ]:
book_name=df_cf_agg.iloc[0,1]
cf_values = df_cf_agg.iloc[0, 2:].tolist()
relevant_months=df_cf_agg.columns[2:].tolist()
df_agg_cf_format=pd.DataFrame({'collection':cf_values,'month':relevant_months})
df_agg_cf_format['month_cohort']=range(1, len(df_agg_cf_format)+1)
cashflow=[df_purch.iloc[0,1]]+df_agg_cf_format['collection'].tolist()
cashflow=[round(value, 2) for value in cashflow]
####Calculating dynamic IRR
irr=[]
selected_values=[cashflow[0]]
for i in range(1, len(cashflow)):
    selected_values.append(cashflow[i])
    irr_temp = round(npf.irr(selected_values),4)
    irr.append(irr_temp)
    
df_agg_cf_format['irr']=irr

#### Calculating Dynamic MOIC
moic_num=0.0
moic_temp=0.0
moic=[]
purchase_value=abs(df_purch.iloc[0,1])
selected_values=[]
for i in range(1, len(cashflow)):
    selected_values.append(cashflow[i])
    moic_num = round((sum(selected_values)),2)
    moic_temp=round(moic_num/purchase_value,2)
    moic.append(moic_temp)
    
df_agg_cf_format['moic']=moic 

df_agg_cf_format['book_id']=book_id
df_agg_cf_format['book_name']=book_name
df_agg_cf_format['purchase_local_ccy']=abs(df_purch.iloc[0,1])
df_agg_cf_format['region']=region.upper()
df_agg_cf_format['purchase_usd']=round(df_purch.iloc[0,3],2)
df_agg_cf_format.head()

In [ ]:
plt.figure(figsize=(10, 6))  # Make the figure wider
plt.scatter(df_agg_cf_format['month_cohort'], df_agg_cf_format['irr'], color='green',s=12)

# Add a line connecting the dots
plt.plot(df_agg_cf_format['month_cohort'], df_agg_cf_format['irr'], color='green', linestyle='-', linewidth=1)

# Add labels and title
plt.xlabel('month Cohort')
plt.ylabel('IRR',fontsize=14)
plt.suptitle(book_name, fontsize=12)
plt.title('Capex (USD) = {:,.0f} '.format(round(df_purch.iloc[0,3]),0), fontsize=10)

# Rotate x-axis labels for better readability (optional)

#x_labels = df_agg_cf_format['month_cohort'].dt.strftime('%Y-%m-%d')  # Format as string
#short_x_labels = x_labels.str.slice(0, 7)  # Get the first 6 letters

plt.xticks(df_agg_cf_format['month_cohort'], rotation=45)


# Annotate each point with its y-value 
#for i, value in enumerate(df_agg_cf_format['irr']):
#    plt.text(df_agg_cf_format['month_cohort'][i], df_agg_cf_format['irr'][i], 
#             f'{value:,.2f}', ha='center', va='bottom', fontsize=9)


#plt.axhline(y=df_average, color='red', linestyle='--', label=f'Average: {df_average:,.2f}')

#plt.text(df_agg_cf_format['month_cohort'].min(), df_average, f'Avg: {df_average:,.2f}', 
 #        color='red', va='bottom', ha='right', fontsize=10)

# Get the last value for annotation
last_x = df_agg_cf_format['month_cohort'].iloc[-1]
last_y = df_agg_cf_format['irr'].iloc[-1]
# Add a label for the last point
plt.text(last_x, last_y, f'{last_y:.2f}', ha='center', va='bottom', fontsize=9, color='green')

# Add a line where y = 0
plt.axhline(y=0, color='red', linestyle='--', label='y = 0')
# Add a label for the y = 0 line
plt.text(df_agg_cf_format['month_cohort'].min(), 0, '0', color='red', va='bottom', ha='center', fontsize=10)


# Add a line where y = 0.48
plt.axhline(y=0.48, color='gray', linestyle='--', label='y = 0.48')
# Add a label for the y = 0 line
plt.text(df_agg_cf_format['month_cohort'].min(), 0.48, 'IRR=48%', color='gray', va='bottom', ha='center', fontsize=10)

# Show plot
plt.tight_layout()  
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))  # Make the figure wider
plt.scatter(df_agg_cf_format['month_cohort'], df_agg_cf_format['moic'], color='blue',s=12)

# Add a line connecting the dots
plt.plot(df_agg_cf_format['month_cohort'], df_agg_cf_format['moic'], color='blue', linestyle='-', linewidth=1)

# Add labels and title
plt.xlabel('month Cohort')
plt.ylabel('MOIC',fontsize=14)
plt.suptitle(book_name, fontsize=12)
plt.title('Capex (USD) = {:,.0f} '.format(round(df_purch.iloc[0,3]),0), fontsize=10)

# Rotate x-axis labels for better readability (optional)

#x_labels = df_agg_cf_format['month_cohort'].dt.strftime('%Y-%m-%d')  # Format as string
#short_x_labels = x_labels.str.slice(0, 7)  # Get the first 6 letters

plt.xticks(df_agg_cf_format['month_cohort'], rotation=45)


# Annotate each point with its y-value 
#for i, value in enumerate(df_agg_cf_format['moic']):
#    plt.text(df_agg_cf_format['month_cohort'][i], df_agg_cf_format['moic'][i], 
#            f'{value:,.2f}', ha='center', va='bottom', fontsize=9)


#plt.axhline(y=df_average, color='red', linestyle='--', label=f'Average: {df_average:,.2f}')

#plt.text(df_agg_cf_format['month_cohort'].min(), df_average, f'Avg: {df_average:,.2f}', 
 #        color='red', va='bottom', ha='right', fontsize=10)

# Get the last value for annotation
last_x = df_agg_cf_format['month_cohort'].iloc[-1]
last_y = df_agg_cf_format['moic'].iloc[-1]
# Add a label for the last point
plt.text(last_x, last_y, f'{last_y:.2f}', ha='center', va='bottom', fontsize=9,color='blue')

# Add a line where y = 0
plt.axhline(y=0, color='red', linestyle='--', label='y = 0')
# Add a label for the y = 0 line
plt.text(df_agg_cf_format['month_cohort'].min(), 0, '0', color='red', va='bottom', ha='center', fontsize=10)


# Add a line where y = 2.25
plt.axhline(y=2.25, color='gray', linestyle='--', label='y = 2.25')
# Add a label for the y = 0 line
plt.text(df_agg_cf_format['month_cohort'].min(), 2.25, 'MOIC=2.25', color='gray', va='bottom', ha='center', fontsize=10)

# Show plot
plt.tight_layout()  
plt.show()

## Pagantis ES Sep 2023

In [ ]:
###Manual: adjust book_id and region
book_id='5176948179'
region='eu' ###'au' or 'eu'

In [ ]:
query_purchase='''SELECT book_id,actual_purchase_price*-1 as purchase_local_ccy,settlement_date,actual_purchase_price_usd
                FROM presentation.dim_books db
                where 1=1
                and book_id='{}' '''.format(book_id)
				
df_purch = pandas_gbq.read_gbq(query_purchase,project_id="data-team-318806")

if region=='au':
    query_cf='''select * except (market, dca_name) from
            presentation.collections_au
            where 1=1
            and book_id='{}'  '''.format(book_id)
elif region=='eu':
    query_cf='''select * except (market, dca_name) from
            presentation.collections_eu
            where 1=1
            and book_id='{}'  '''.format(book_id)
else:
    query_cf=None


df_cf=pandas_gbq.read_gbq(query_cf,project_id="data-team-318806")

df_cf_agg = df_cf.groupby(['book_id', 'book_name']).sum().reset_index().copy()

In [ ]:
####Removing as of 3rd row the zero values until first non zero
# Find the index of the first non-zero column starting from the third column
non_zero_columns = df_cf_agg.columns[2:]  # Only columns from C onwards
first_non_zero_col = next((col for col in non_zero_columns if df_cf_agg[col].iloc[0] != 0), None)

# Identify columns to drop: from the third column up to the first non-zero column
if first_non_zero_col:
    columns_to_drop = []
    for col in non_zero_columns:
        if col == first_non_zero_col:
            break
        if df_cf_agg[col].iloc[0] == 0.0:
            columns_to_drop.append(col)
    df_cf_agg.drop(columns=columns_to_drop, inplace=True)

In [ ]:
book_name=df_cf_agg.iloc[0,1]
cf_values = df_cf_agg.iloc[0, 2:].tolist()
relevant_months=df_cf_agg.columns[2:].tolist()
df_agg_cf_format=pd.DataFrame({'collection':cf_values,'month':relevant_months})
df_agg_cf_format['month_cohort']=range(1, len(df_agg_cf_format)+1)
cashflow=[df_purch.iloc[0,1]]+df_agg_cf_format['collection'].tolist()
cashflow=[round(value, 2) for value in cashflow]
####Calculating dynamic IRR
irr=[]
selected_values=[cashflow[0]]
for i in range(1, len(cashflow)):
    selected_values.append(cashflow[i])
    irr_temp = round(npf.irr(selected_values),4)
    irr.append(irr_temp)
    
df_agg_cf_format['irr']=irr

#### Calculating Dynamic MOIC
moic_num=0.0
moic_temp=0.0
moic=[]
purchase_value=abs(df_purch.iloc[0,1])
selected_values=[]
for i in range(1, len(cashflow)):
    selected_values.append(cashflow[i])
    moic_num = round((sum(selected_values)),2)
    moic_temp=round(moic_num/purchase_value,2)
    moic.append(moic_temp)
    
df_agg_cf_format['moic']=moic 

df_agg_cf_format['book_id']=book_id
df_agg_cf_format['book_name']=book_name
df_agg_cf_format['purchase_local_ccy']=abs(df_purch.iloc[0,1])
df_agg_cf_format['region']=region.upper()
df_agg_cf_format['purchase_usd']=round(df_purch.iloc[0,3],2)
df_agg_cf_format.head()

In [ ]:
plt.figure(figsize=(10, 6))  # Make the figure wider
plt.scatter(df_agg_cf_format['month_cohort'], df_agg_cf_format['irr'], color='green',s=12)

# Add a line connecting the dots
plt.plot(df_agg_cf_format['month_cohort'], df_agg_cf_format['irr'], color='green', linestyle='-', linewidth=1)

# Add labels and title
plt.xlabel('month Cohort')
plt.ylabel('IRR',fontsize=14)
plt.suptitle(book_name, fontsize=12)
plt.title('Capex (USD) = {:,.0f} '.format(round(df_purch.iloc[0,3]),0), fontsize=10)

# Rotate x-axis labels for better readability (optional)

#x_labels = df_agg_cf_format['month_cohort'].dt.strftime('%Y-%m-%d')  # Format as string
#short_x_labels = x_labels.str.slice(0, 7)  # Get the first 6 letters

plt.xticks(df_agg_cf_format['month_cohort'], rotation=45)


# Annotate each point with its y-value 
#for i, value in enumerate(df_agg_cf_format['irr']):
#    plt.text(df_agg_cf_format['month_cohort'][i], df_agg_cf_format['irr'][i], 
#             f'{value:,.2f}', ha='center', va='bottom', fontsize=9)


#plt.axhline(y=df_average, color='red', linestyle='--', label=f'Average: {df_average:,.2f}')

#plt.text(df_agg_cf_format['month_cohort'].min(), df_average, f'Avg: {df_average:,.2f}', 
 #        color='red', va='bottom', ha='right', fontsize=10)

# Get the last value for annotation
last_x = df_agg_cf_format['month_cohort'].iloc[-1]
last_y = df_agg_cf_format['irr'].iloc[-1]
# Add a label for the last point
plt.text(last_x, last_y, f'{last_y:.2f}', ha='center', va='bottom', fontsize=9, color='green')

# Add a line where y = 0
plt.axhline(y=0, color='red', linestyle='--', label='y = 0')
# Add a label for the y = 0 line
plt.text(df_agg_cf_format['month_cohort'].min(), 0, '0', color='red', va='bottom', ha='center', fontsize=10)


# Add a line where y = 0.48
plt.axhline(y=0.48, color='gray', linestyle='--', label='y = 0.48')
# Add a label for the y = 0 line
plt.text(df_agg_cf_format['month_cohort'].min(), 0.48, 'IRR=48%', color='gray', va='bottom', ha='center', fontsize=10)

# Show plot
plt.tight_layout()  
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))  # Make the figure wider
plt.scatter(df_agg_cf_format['month_cohort'], df_agg_cf_format['moic'], color='blue',s=12)

# Add a line connecting the dots
plt.plot(df_agg_cf_format['month_cohort'], df_agg_cf_format['moic'], color='blue', linestyle='-', linewidth=1)

# Add labels and title
plt.xlabel('month Cohort')
plt.ylabel('MOIC',fontsize=14)
plt.suptitle(book_name, fontsize=12)
plt.title('Capex (USD) = {:,.0f} '.format(round(df_purch.iloc[0,3]),0), fontsize=10)

# Rotate x-axis labels for better readability (optional)

#x_labels = df_agg_cf_format['month_cohort'].dt.strftime('%Y-%m-%d')  # Format as string
#short_x_labels = x_labels.str.slice(0, 7)  # Get the first 6 letters

plt.xticks(df_agg_cf_format['month_cohort'], rotation=45)


# Annotate each point with its y-value 
#for i, value in enumerate(df_agg_cf_format['moic']):
#    plt.text(df_agg_cf_format['month_cohort'][i], df_agg_cf_format['moic'][i], 
#            f'{value:,.2f}', ha='center', va='bottom', fontsize=9)


#plt.axhline(y=df_average, color='red', linestyle='--', label=f'Average: {df_average:,.2f}')

#plt.text(df_agg_cf_format['month_cohort'].min(), df_average, f'Avg: {df_average:,.2f}', 
 #        color='red', va='bottom', ha='right', fontsize=10)

# Get the last value for annotation
last_x = df_agg_cf_format['month_cohort'].iloc[-1]
last_y = df_agg_cf_format['moic'].iloc[-1]
# Add a label for the last point
plt.text(last_x, last_y, f'{last_y:.2f}', ha='center', va='bottom', fontsize=9,color='blue')

# Add a line where y = 0
plt.axhline(y=0, color='red', linestyle='--', label='y = 0')
# Add a label for the y = 0 line
plt.text(df_agg_cf_format['month_cohort'].min(), 0, '0', color='red', va='bottom', ha='center', fontsize=10)


# Add a line where y = 2.25
plt.axhline(y=2.25, color='gray', linestyle='--', label='y = 2.25')
# Add a label for the y = 0 line
plt.text(df_agg_cf_format['month_cohort'].min(), 2.25, 'MOIC=2.25', color='gray', va='bottom', ha='center', fontsize=10)

# Show plot
plt.tight_layout()  
plt.show()

## IPF AU Sep 2023

In [ ]:
###Manual: adjust book_id and region
book_id='5152696966'
region='au' ###'au' or 'eu'

In [ ]:
query_purchase='''SELECT book_id,actual_purchase_price*-1 as purchase_local_ccy,settlement_date,actual_purchase_price_usd
                FROM presentation.dim_books db
                where 1=1
                and book_id='{}' '''.format(book_id)
				
df_purch = pandas_gbq.read_gbq(query_purchase,project_id="data-team-318806")

if region=='au':
    query_cf='''select * except (market, dca_name) from
            presentation.collections_au
            where 1=1
            and book_id='{}'  '''.format(book_id)
elif region=='eu':
    query_cf='''select * except (market, dca_name) from
            presentation.collections_eu
            where 1=1
            and book_id='{}'  '''.format(book_id)
else:
    query_cf=None


df_cf=pandas_gbq.read_gbq(query_cf,project_id="data-team-318806")

df_cf_agg = df_cf.groupby(['book_id', 'book_name']).sum().reset_index().copy()

In [ ]:
####Removing as of 3rd row the zero values until first non zero
# Find the index of the first non-zero column starting from the third column
non_zero_columns = df_cf_agg.columns[2:]  # Only columns from C onwards
first_non_zero_col = next((col for col in non_zero_columns if df_cf_agg[col].iloc[0] != 0), None)

# Identify columns to drop: from the third column up to the first non-zero column
if first_non_zero_col:
    columns_to_drop = []
    for col in non_zero_columns:
        if col == first_non_zero_col:
            break
        if df_cf_agg[col].iloc[0] == 0.0:
            columns_to_drop.append(col)
    df_cf_agg.drop(columns=columns_to_drop, inplace=True)

In [ ]:
book_name=df_cf_agg.iloc[0,1]
cf_values = df_cf_agg.iloc[0, 2:].tolist()
relevant_months=df_cf_agg.columns[2:].tolist()
df_agg_cf_format=pd.DataFrame({'collection':cf_values,'month':relevant_months})
df_agg_cf_format['month_cohort']=range(1, len(df_agg_cf_format)+1)
cashflow=[df_purch.iloc[0,1]]+df_agg_cf_format['collection'].tolist()
cashflow=[round(value, 2) for value in cashflow]
####Calculating dynamic IRR
irr=[]
selected_values=[cashflow[0]]
for i in range(1, len(cashflow)):
    selected_values.append(cashflow[i])
    irr_temp = round(npf.irr(selected_values),4)
    irr.append(irr_temp)
    
df_agg_cf_format['irr']=irr

#### Calculating Dynamic MOIC
moic_num=0.0
moic_temp=0.0
moic=[]
purchase_value=abs(df_purch.iloc[0,1])
selected_values=[]
for i in range(1, len(cashflow)):
    selected_values.append(cashflow[i])
    moic_num = round((sum(selected_values)),2)
    moic_temp=round(moic_num/purchase_value,2)
    moic.append(moic_temp)
    
df_agg_cf_format['moic']=moic 

df_agg_cf_format['book_id']=book_id
df_agg_cf_format['book_name']=book_name
df_agg_cf_format['purchase_local_ccy']=abs(df_purch.iloc[0,1])
df_agg_cf_format['region']=region.upper()
df_agg_cf_format['purchase_usd']=round(df_purch.iloc[0,3],2)
df_agg_cf_format.head()

In [ ]:
plt.figure(figsize=(10, 6))  # Make the figure wider
plt.scatter(df_agg_cf_format['month_cohort'], df_agg_cf_format['irr'], color='green',s=12)

# Add a line connecting the dots
plt.plot(df_agg_cf_format['month_cohort'], df_agg_cf_format['irr'], color='green', linestyle='-', linewidth=1)

# Add labels and title
plt.xlabel('month Cohort')
plt.ylabel('IRR',fontsize=14)
plt.suptitle(book_name, fontsize=12)
plt.title('Capex (USD) = {:,.0f} '.format(round(df_purch.iloc[0,3]),0), fontsize=10)

# Rotate x-axis labels for better readability (optional)

#x_labels = df_agg_cf_format['month_cohort'].dt.strftime('%Y-%m-%d')  # Format as string
#short_x_labels = x_labels.str.slice(0, 7)  # Get the first 6 letters

plt.xticks(df_agg_cf_format['month_cohort'], rotation=45)


# Annotate each point with its y-value 
#for i, value in enumerate(df_agg_cf_format['irr']):
#    plt.text(df_agg_cf_format['month_cohort'][i], df_agg_cf_format['irr'][i], 
#             f'{value:,.2f}', ha='center', va='bottom', fontsize=9)


#plt.axhline(y=df_average, color='red', linestyle='--', label=f'Average: {df_average:,.2f}')

#plt.text(df_agg_cf_format['month_cohort'].min(), df_average, f'Avg: {df_average:,.2f}', 
 #        color='red', va='bottom', ha='right', fontsize=10)

# Get the last value for annotation
last_x = df_agg_cf_format['month_cohort'].iloc[-1]
last_y = df_agg_cf_format['irr'].iloc[-1]
# Add a label for the last point
plt.text(last_x, last_y, f'{last_y:.2f}', ha='center', va='bottom', fontsize=9, color='green')

# Add a line where y = 0
plt.axhline(y=0, color='red', linestyle='--', label='y = 0')
# Add a label for the y = 0 line
plt.text(df_agg_cf_format['month_cohort'].min(), 0, '0', color='red', va='bottom', ha='center', fontsize=10)


# Add a line where y = 0.48
plt.axhline(y=0.48, color='gray', linestyle='--', label='y = 0.48')
# Add a label for the y = 0 line
plt.text(df_agg_cf_format['month_cohort'].min(), 0.48, 'IRR=48%', color='gray', va='bottom', ha='center', fontsize=10)

# Show plot
plt.tight_layout()  
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))  # Make the figure wider
plt.scatter(df_agg_cf_format['month_cohort'], df_agg_cf_format['moic'], color='blue',s=12)

# Add a line connecting the dots
plt.plot(df_agg_cf_format['month_cohort'], df_agg_cf_format['moic'], color='blue', linestyle='-', linewidth=1)

# Add labels and title
plt.xlabel('month Cohort')
plt.ylabel('MOIC',fontsize=14)
plt.suptitle(book_name, fontsize=12)
plt.title('Capex (USD) = {:,.0f} '.format(round(df_purch.iloc[0,3]),0), fontsize=10)

# Rotate x-axis labels for better readability (optional)

#x_labels = df_agg_cf_format['month_cohort'].dt.strftime('%Y-%m-%d')  # Format as string
#short_x_labels = x_labels.str.slice(0, 7)  # Get the first 6 letters

plt.xticks(df_agg_cf_format['month_cohort'], rotation=45)


# Annotate each point with its y-value 
#for i, value in enumerate(df_agg_cf_format['moic']):
#    plt.text(df_agg_cf_format['month_cohort'][i], df_agg_cf_format['moic'][i], 
#            f'{value:,.2f}', ha='center', va='bottom', fontsize=9)


#plt.axhline(y=df_average, color='red', linestyle='--', label=f'Average: {df_average:,.2f}')

#plt.text(df_agg_cf_format['month_cohort'].min(), df_average, f'Avg: {df_average:,.2f}', 
 #        color='red', va='bottom', ha='right', fontsize=10)

# Get the last value for annotation
last_x = df_agg_cf_format['month_cohort'].iloc[-1]
last_y = df_agg_cf_format['moic'].iloc[-1]
# Add a label for the last point
plt.text(last_x, last_y, f'{last_y:.2f}', ha='center', va='bottom', fontsize=9,color='blue')

# Add a line where y = 0
plt.axhline(y=0, color='red', linestyle='--', label='y = 0')
# Add a label for the y = 0 line
plt.text(df_agg_cf_format['month_cohort'].min(), 0, '0', color='red', va='bottom', ha='center', fontsize=10)


# Add a line where y = 2.25
plt.axhline(y=2.25, color='gray', linestyle='--', label='y = 2.25')
# Add a label for the y = 0 line
plt.text(df_agg_cf_format['month_cohort'].min(), 2.25, 'MOIC=2.25', color='gray', va='bottom', ha='center', fontsize=10)

# Show plot
plt.tight_layout()  
plt.show()